In [8]:
import time
import os
import shutil
import csv
import torch
import numpy as np
import math
# import itertools
from itertools import product
import torch.nn.functional as F
import pandas as pd
from scipy.stats import poisson
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("CUDA AVAILABLE" if device=='cuda' else "cuda NOT available")

cuda NOT available


In [9]:
def soft_shift_poisson(rate:int, left_shift:int=0) -> float:
    """
    Returns the mathematical expectation of max(0,X-left_shift), where X a Poisson distribution.
    """
    if left_shift == 0: return rate
    sub_probability = poisson.cdf(2*left_shift-1,rate)
    sub_mean =  0
    for n in range(2*left_shift): sub_mean += n*poisson.pmf(n,rate)
    remaining_mean = rate - sub_mean
    remaining_probability = 1 - sub_probability
    return remaining_mean - remaining_probability*left_shift

In [16]:
class ground_model:
    """
    Definition and operations for a radio resource allocation problem.
    Parameters:
    max_size = maximum number of bits in an user equipment's queue;
    number of user equipments; number of resource blocks;
    default_arrival_rates: indicate if the arrival rate at each user equipment's buffer is set to default
    CQIs_are_equal: indicates if the chanel quality index is constant in the UE and in the RB;
    CQI_base = number of bits scheduled for transmission when CQI=1;
    coef_of_drop, coef_of_latency, power_of_drop, power_of_latency = α, β, x, y:
    cost = α*(cost for rejection)**x + β(cost for latency)**y;
    discount factor;
    precision = bound of the difference between the calculated and the optimal value function of the MDP;
    path = location of the results of the operations (if it is not indicated from the root, the location is in the current directory).
    """
    
    def __init__(self, max_size:int=3, number_UEs:int=2, number_RBs:int=2, 
                 default_arrival_rates=None,
                 CQIs_are_equal=True, CQI_base:float=1,
                 coef_of_drop:float=1, coef_of_latency:float=1, power_of_drop:float=1, power_of_latency:float=1, 
                 discount_factor:float=0.9, precision:float=1e-6,
                 path=None) -> None:
        
        # States and actions
        number_states, number_actions = (max_size + 1) ** number_UEs, number_UEs ** number_RBs
        range_UE_indices = torch.tensor(range(number_UEs))
        range_number_bits = torch.tensor(range(max_size+1))
        list_states = list(product(range_number_bits,repeat=number_UEs))
        list_actions = list(product(range_UE_indices, repeat=number_RBs))
        
        # Send to the device: the data, the ranges for indices, the states and the actions
        self.max_size = torch.tensor([max_size],dtype=int).to(device)
        self.number_UEs, self.number_RBs = torch.tensor([number_UEs],dtype=int).to(device), torch.tensor([number_RBs],dtype=int).to(device)
        self.coef_of_drop, self.coef_of_latency = torch.tensor([coef_of_drop],dtype=float).to(device), torch.tensor([coef_of_latency],dtype=float).to(device)
        self.power_of_drop, self.power_of_latency = torch.tensor([power_of_drop],dtype=float).to(device), torch.tensor([power_of_latency],dtype=float).to(device)
        self.discount_factor, self.precision = torch.tensor([discount_factor], dtype=float).to(device), torch.tensor([precision], dtype=float).to(device)
        self.number_states, self.number_actions = torch.tensor([number_states]).to(device), torch.tensor([number_actions]).to(device)
        self.range_UE_indices, self.range_RB_indices = torch.tensor(range(number_UEs)).to(device), torch.tensor(range(number_RBs)).to(device)
        self.range_state_indices, self.range_action_indices = torch.tensor(range(number_states)).to(device), torch.tensor(range(number_actions)).to(device)
        self.matrix_states, self.matrix_actions = torch.tensor(list_states).to(device), torch.tensor(list_actions).to(device)
        self.empty_buffer = torch.zeros([self.number_UEs],dtype=int).to(device)                      # Number of bits in each UE's empty buffer
        self.full_buffer = self.max_size*torch.ones([self.number_UEs],dtype=int).to(device)          # Number of bits in each UE's full buffer
        
        # Get the arrival rates
        if default_arrival_rates==None:
            vector_arrival_rates = torch.ones([self.number_UEs]).to(device)
            fraction = torch.tensor([max_size/3],dtype=float).to(device)
            print(f"Arrival rate is set to default for each UE: lambda = {fraction.item()}")
            vector_arrival_rates = fraction*vector_arrival_rates
        else:
            vector_arrival_rates = torch.ones([self.number_UEs]).to(device)
            print("Setting the arrival rate for each UE.")
            decision = input("Same arrival rate? (y/n)")
            if "Y" in decision.upper():
                value_ = float(input("For all UE, arrival rate is: "))
                value_ = torch.tensor([value_],dtype=float).to(device)
                vector_arrival_rates = value_*vector_arrival_rates
            else:
                for i in self.range_UE_indices:
                    print(f"Arrival rate for UE {i}: ")
                    vector_arrival_rates[i] = float(input())
        self.vector_arrival_rates = vector_arrival_rates
        
        # Get the chanel quality indices matrix
        if CQIs_are_equal:
            self.CQI_matrix = CQI_base*torch.ones([self.number_UEs,number_RBs],dtype=float).to(device)
        else:
            self.CQI_matrix = torch.ones([self.number_UEs,number_RBs],dtype=float)
            print("Definition of the chanel quality indices.")
            decision = input("Allow various values of CQI? (y/n): ")
            if "Y" in decision.upper():
                print("\nFor each UE and each RB, let's define the CQI as:  CQI = (coef of UE and RB)*(common base)")
                print("Enter all the coefficients. Later on, you will enter the common base to multiply.")
                for i,j in product(self.range_UE_indices,self.range_RB_indices):
                    print(f"    Coefficient for UE {i} and RB {j}: ", end="")
                    value_ = input()
                    if value_.isnumeric(): 
                        value_ = torch.tensor([value_],dtype=int).to(device)
                        self.CQI_matrix[i,j] = value_
                value_ = input("Enter the common base: ")
                if value_.isnumeric(): 
                    value_ = torch.tensor([value_],dtype=int).to(device)
                    CQI_base = value_
                self.CQI_matrix = CQI_base*self.CQI_matrix
                self.CQI_matrix = self.CQI_matrix.to(device)
                
        # Prepare the resolution
        self.prepare_resolution()
        
    def prepare_resolution(self, directory_root=None, solution=None):
        
        # Prepare the path for the solution and the simulations
        if directory_root == None:
            this_directory = os.getcwd()
            solution_directory = "Solution_for_B-" + str(self.max_size.item()) + "_UE-" + str(self.number_UEs.item()) + "_RB-" + str(self.number_RBs.item())
            directory_root = os.path.join(this_directory, solution_directory)
        last_index = 1
        while True:
            directory = (directory_root if last_index==1 else directory_root + "_"+str(last_index))
            if os.path.exists(directory):
                print(f"Directory {directory} serves for the resolution.")
                decision = input("Continue the last resolution in it? (Y/N): ")
                if "Y" in decision.upper(): break
                else:
                    print(f"This is, you should decide wether delete the existing resolution in {directory}")
                    print("and start a new resolution in this directory or let me look for another directory.")
                    decision = input("Start a new resolution in it? (Y/N): ")
                    if "Y" in decision.upper():
                        shutil.rmtree(directory)
                        os.makedirs(directory)
                        break
                    else: last_index += 1
            else:
                os.makedirs(directory)
                break
        self.directory = directory
        
        # Prepare the initial value for the backup iteration and save it in a dedicated file
        solution_path = os.path.join(self.directory, "ground_solution.pth")
        if os.path.exists(solution_path):
            if solution == None: self.solution = torch.load(solution_path).to(device)
            else:
                self.solution = solution
                torch.save(self.solution, solution_path)
        else:
            self.solution = (torch.zeros([2,self.number_states],dtype=float).to(device) if solution==None else solution)
            torch.save(self.solution, solution_path)
        self.solution_path = solution_path
        
    
    def remainder_fn(self, state:torch.tensor, action:torch.tensor) -> torch.tensor:
        """
        Returns the number of bits remaining in each UE's buffer if action of index action_index was taken in state of index state_index.
        """
        schedule = torch.zeros([self.number_UEs],dtype=float).to(device)  # Number of bits scheduled for transmission
        for i,j in product(self.range_UE_indices,self.range_RB_indices): 
            if action[j]==i:
                schedule[i] += self.CQI_matrix[i,j]
        schedule = schedule.to(int)
        angry_remainder = state - schedule    # Number of remaining bits if negative values are possible
        return torch.max(self.empty_buffer, angry_remainder)
    
    def transition_probability_fn(self, state1:torch.tensor, action:torch.tensor, state2:torch.tensor) -> torch.tensor:
        def UE_transition_probability_fn(rest:int, rate:float, next:int) -> torch.tensor:
            max_size = self.max_size
            sure, impossible = torch.tensor([1],dtype=float).to(device), torch.tensor([0],dtype=float).to(device)
            if next<rest:     return impossible                                                           # next < rest
            if next<max_size: return torch.tensor([poisson.pmf(next-rest,rate)],dtype=float).to(device)   # rest ≤ next < max_size  
            range_next = torch.tensor(range((max_size-rest).item()),dtype=int).to(device)                 # next = max_size, then  P{size=next} = 1 - P{arrival<}
            probability = sure
            for arrival in range_next:
                probability_arrival = torch.tensor([poisson.pmf(arrival,rate)],dtype=float).to(device)
                probability = probability - probability_arrival
            return probability
        rate = self.vector_arrival_rates
        remainder = self.remainder_fn(state1,action)
        probability = torch.tensor([1],dtype=float).to(device)
        for i in self.range_UE_indices:                                                                    # i = queue index
            probability = probability * UE_transition_probability_fn(remainder[i], rate[i], state2[i])
        return probability
    
    def cost_fn(self, state:torch.tensor, action:torch.tensor) -> torch.tensor:
        def exact_cost_fn(remainder:torch.tensor, state2:torch.tensor) -> torch.tensor:                     # Exact cost associated with the transition from state1 to state2
            def partial_cost_fn(rest:torch.tensor, arrival:torch.tensor) -> torch.tensor:                   # Cost for transition of the queue
                false_excess = arrival + rest + self.max_size
                excess = torch.max(nothing, false_excess)
                return self.coef_of_drop * excess**self.power_of_drop + self.coef_of_latency * rest**self.power_of_latency
            exact_cost = torch.tensor([0],dtype=float).to(device)
            for q in self.range_UE_indices:
                if state2[q] < remainder[q]:                                                                 # The transition is impossible
                    exact_cost = torch.tensor([0],dtype=float).to(device)
                    break
                else:
                    arrival = state2[q] - remainder[q]
                    exact_cost = exact_cost + partial_cost_fn(remainder[q], arrival)
            return exact_cost
        nothing = torch.tensor([0],dtype=int).to(device)
        remainder = self.remainder_fn(state, action)
        cost = torch.tensor([0],dtype=float).to(device)
        for s in self.range_state_indices:
            possible_state = self.matrix_states[s]
            possible_exact_cost = exact_cost_fn(remainder, possible_state)
            cost = cost + possible_exact_cost * self.transition_probability_fn(state,action,possible_state)
        return cost
    
    def resolution(self, directory=None, solution=None, precision=None) -> torch.tensor:
        
        # Initialization
        if directory == None: directory = self.directory
        if precision == None: precision = self.precision
        self.prepare_resolution(directory, solution)
        
        # Body of the resolution
        gamma = self.discount_factor.item()
        fraction = gamma / (1-gamma)
        old_value = new_value = self.solution[1].to(device)
        policy = torch.zeros([self.number_states],dtype=int).to(device)
        try:
            while True:
                for s in self.range_state_indices:
                    state1 = self.matrix_states[s]
                    Q = torch.zeros([self.number_actions],dtype=float).to(device)
                    for a in self.range_action_indices:
                        action = self.matrix_actions[a]
                        P = torch.zeros([self.number_states],dtype=float).to(device)
                        for s2 in self.range_state_indices: 
                            state2 = self.matrix_states[s2]
                            P[s2] = self.transition_probability_fn(state1, action, state2)
                        Q[a] = self.cost_fn(state1,action) + self.discount_factor*torch.sum(old_value*P)
                    new_value[s] = torch.min(Q)
                    policy[s] = torch.argmin(Q)
                current_policy, current_value = policy, new_value
                min_diff, max_diff = torch.min(new_value - old_value).item(), torch.max(new_value - old_value).item() 
                bound1, bound2 = fraction*min_diff, fraction*max_diff
                error = max((bound1, bound2))
                if error <= precision: break
        except KeyboardInterrupt:
            pass 
        
        # Storing the solution
        self.solution = torch.stack([current_policy, current_value])
        torch.save(self.solution, self.solution_path)

In [17]:
# TEST FOR TRANSITION PROBABILITIES

model = ground_model()
max_diff = 0
for ai in range(model.number_actions):
    AT = model.matrix_actions[ai]
    AN = AT.numpy()
    print(f"\n\nWith action a_{ai} = {AN}\n")
    for si1 in range(model.number_states):
        S1T = model.matrix_states[si1]
        S1N = S1T.numpy()
        S = torch.tensor([0]).to(device)
        for si2 in range(model.number_states):
            S2T = model.matrix_states[si2]
            PT = model.transition_probability_fn(S1T,AT,S2T)
            S = S + PT
            S2N, PN = S2T.numpy(), PT.item()
            print(f"a_{ai}:      {S1N} --->  {S2N}:    P = {PN}")
        SN = S.item()
        print(f"\n                  Sum of probabilities = {SN}\n\n")
        max_diff = max((max_diff, abs(SN-1)))
print(f"\nMax error = {max_diff}")

Arrival rate is set to default for each UE: lambda = 1.0
Directory /home/tsemogne/Documents/Postdoc/Radio Resour Allocation/Solution_for_B-3_UE-2_RB-2 serves for the resolution.


With action a_0 = [0 0]

a_0:      [0 0] --->  [0 0]:    P = 0.13533528323661273
a_0:      [0 0] --->  [0 1]:    P = 0.13533528323661273
a_0:      [0 0] --->  [0 2]:    P = 0.06766764161830637
a_0:      [0 0] --->  [0 3]:    P = 0.029541233079910564
a_0:      [0 0] --->  [1 0]:    P = 0.13533528323661273
a_0:      [0 0] --->  [1 1]:    P = 0.13533528323661273
a_0:      [0 0] --->  [1 2]:    P = 0.06766764161830637
a_0:      [0 0] --->  [1 3]:    P = 0.029541233079910564
a_0:      [0 0] --->  [2 0]:    P = 0.06766764161830637
a_0:      [0 0] --->  [2 1]:    P = 0.06766764161830637
a_0:      [0 0] --->  [2 2]:    P = 0.033833820809153176
a_0:      [0 0] --->  [2 3]:    P = 0.01477061653995528
a_0:      [0 0] --->  [3 0]:    P = 0.029541233079910564
a_0:      [0 0] --->  [3 1]:    P = 0.029541233079910564
a_0:  

In [18]:
model.resolution()

Directory /home/tsemogne/Documents/Postdoc/Radio Resour Allocation/Solution_for_B-3_UE-2_RB-2 serves for the resolution.


In [19]:
model.solution

tensor([[ 0.0000,  1.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000,
          3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  1.0000,  1.0000],
        [ 7.9533,  8.9221, 10.0088, 12.1788,  9.8730, 11.0756, 12.4246, 14.6551,
         11.6088, 13.0228, 14.6089, 16.8941, 12.9767, 17.2731, 21.9872, 25.1811]],
       dtype=torch.float64)

In [57]:
# TEST OF COSTS

list_rests = list_costs = []
for a,s in product(model.range_action_indices,model.range_state_indices):
    action, state = model.matrix_actions[a], model.matrix_states[s]
    remainder = model.remainder_fn(state,action)
    cost = model.cost_fn(state, action)
    list_rests.append(torch.sum(remainder).item())
    list_costs.append(cost.item())
    print(f"Action {action.numpy()}  in state {state.numpy()}      --->      Rest = {remainder.numpy()}  and Cost = {cost.item()}")
print(f"\nyThe correlation coefficient between the cost and the sum of the rest in queues is {np.corrcoef(list_costs,list_rests)[0,1]}")

Action [0 0]  in state [0 0]      --->      Rest = [0 0]  and Cost = 7.9533261471141365
Action [0 0]  in state [0 1]      --->      Rest = [0 1]  and Cost = 9.873024750042742
Action [0 0]  in state [0 2]      --->      Rest = [0 2]  and Cost = 11.608783632385624
Action [0 0]  in state [0 3]      --->      Rest = [0 3]  and Cost = 12.976663073557066
Action [0 0]  in state [1 0]      --->      Rest = [0 0]  and Cost = 7.9533261471141365
Action [0 0]  in state [1 1]      --->      Rest = [0 1]  and Cost = 9.873024750042742
Action [0 0]  in state [1 2]      --->      Rest = [0 2]  and Cost = 11.608783632385624
Action [0 0]  in state [1 3]      --->      Rest = [0 3]  and Cost = 12.976663073557066
Action [0 0]  in state [2 0]      --->      Rest = [0 0]  and Cost = 7.9533261471141365
Action [0 0]  in state [2 1]      --->      Rest = [0 1]  and Cost = 9.873024750042742
Action [0 0]  in state [2 2]      --->      Rest = [0 2]  and Cost = 11.608783632385624
Action [0 0]  in state [2 3]      -